In [19]:
import pandas as pd
from unidecode import unidecode
import re

In [20]:
# 1. Función para extraer modelos de autos de la columna 'Aplicaciones'
def extraer_autos_de_aplicaciones(aplicaciones):
    if pd.isna(aplicaciones) or aplicaciones == '':
        return []
    
    # Patrones para extraer modelos de autos
    patrones = [
        r'([A-Z][A-Za-z]+ [A-Z][A-Za-z-]+ \d{4}/\d{4})',  # Ej: "CORSA 1.4 1.6 1993/1998"
        r'([A-Z][A-Za-z]+ \d\.\d \d{4}[-\/]\d{4})',  # Ej: "CORSA 1.6 1993-2010"
        r'([A-Z][A-Za-z]+ [A-Z][A-Za-z]+ \d{4}[-\/]\d{4})',  # Ej: "HYUNDAI GRAND I10 2015/2018"
        r'([A-Z][A-Za-z]+ \d{4}[-\/]\d{4})',  # Ej: "SONATA 2006/2010"
        r'([A-Z][A-Za-z]+ [A-Z][A-Za-z-]+)',  # Ej: "CHEVROLET SAIL"
        r'([A-Z][A-Za-z]+ \d\.\d)',  # Ej: "CORSA 1.6"
        r'([A-Z][A-Za-z]+)'  # Solo marca/modelo
    ]
    
    autos_encontrados = set()
    
    for patron in patrones:
        matches = re.findall(patron, str(aplicaciones))
        for match in matches:
            # Limpiar y normalizar el resultado
            auto = re.sub(r'\d+\.\d', '', match)  # Quitar cilindradas
            auto = re.sub(r'\d{4}[-\/]\d{4}', '', auto)  # Quitar años
            auto = re.sub(r'\(.*?\)', '', auto)  # Quitar texto entre paréntesis
            auto = unidecode(auto).upper().strip()
            if len(auto) > 3:  # Filtrar resultados muy cortos
                autos_encontrados.add(auto)
    
    return list(autos_encontrados)

In [21]:
# 2. Función para procesar hojas de repuestos
def procesar_hojas_repuestos(archivo):
    hojas_repuestos = ['F1', 'F2', 'F3']
    datos_autos = []
    
    for hoja in hojas_repuestos:
        try:
            df = pd.read_excel(archivo, sheet_name=hoja)
            if 'Aplicaciones' not in df.columns:
                continue
                
            for _, row in df.iterrows():
                autos = extraer_autos_de_aplicaciones(row['Aplicaciones'])
                for auto in autos:
                    # Separar marca y modelo (asumimos primera palabra es marca)
                    partes = auto.split()
                    if len(partes) >= 2:
                        marca = partes[0]
                        modelo = ' '.join(partes[1:])
                        datos_autos.append({
                            'MARCA': marca,
                            'MODELO': modelo,
                            'FUENTE': f'repuesto_{hoja}',
                            'CODIGO_REPUESTO': row.get('Codigo', '')
                        })
        except:
            continue
    
    return pd.DataFrame(datos_autos)

In [22]:
# 3. Función para procesar hoja de vehículos (F4)
def procesar_hoja_vehiculos(archivo):
    df = pd.read_excel(archivo, sheet_name='F4')
    df = df.dropna(subset=['MARCA', 'MODELO', 'AÑO_FAB'])
    
    # Crear dataframe compatible con el de repuestos
    vehiculos = df[['MARCA', 'MODELO']].copy()
    vehiculos['FUENTE'] = 'vehiculo_F4'
    vehiculos['CODIGO_REPUESTO'] = ''
    
    return vehiculos

In [23]:
# 4. Función principal consolidada
def procesar_archivo_completo(archivo_excel):
    # Procesar vehículos directos
    vehiculos = procesar_hoja_vehiculos(archivo_excel)
    
    # Procesar autos mencionados en repuestos
    autos_repuestos = procesar_hojas_repuestos(archivo_excel)
    
    # Combinar ambos datasets
    todos_autos = pd.concat([vehiculos, autos_repuestos], ignore_index=True)
    
    # Normalización
    todos_autos['MARCA'] = todos_autos['MARCA'].apply(lambda x: unidecode(x).upper().strip())
    todos_autos['MODELO_BASE'] = todos_autos['MODELO'].apply(extraer_modelo_base)
    
    # Eliminar duplicados (misma marca y modelo base)
    todos_autos = todos_autos.drop_duplicates(subset=['MARCA', 'MODELO_BASE'])
    
    # Crear tablas normalizadas
    marcas = todos_autos[['MARCA']].drop_duplicates().reset_index(drop=True)
    marcas['MARCA_ID'] = marcas.index + 1
    
    modelos = todos_autos[['MARCA', 'MODELO_BASE']].drop_duplicates()
    modelos = modelos.merge(marcas, on='MARCA')
    modelos['MODELO_ID'] = modelos.index + 1
    modelos = modelos[['MODELO_ID', 'MARCA_ID', 'MODELO_BASE']]
    
    # Para los autos completos (de la hoja F4)
    if not vehiculos.empty:
        autos_completos = vehiculos.merge(modelos, 
                                       left_on=['MARCA', 'MODELO_BASE'], 
                                       right_on=['MARCA', 'MODELO_BASE'])
        autos_completos = autos_completos[[
            'MARCA_ID', 'MODELO_ID', 'MODELO', 'FUENTE'
        ]]
    else:
        autos_completos = pd.DataFrame()
    
    return marcas, modelos, autos_completos

In [24]:
# Funciones auxiliares (las mismas que antes)
def extraer_modelo_base(texto):
    texto = unidecode(texto).upper()
    patrones_eliminar = [
        r'\d\.\d', r'\d+\.\d+L', r'CVT', r'AT', r'MT',
        r'\dX\d', r'[A-Z]{2,}', r'\d{4}', r'\(.*?\)'
    ]
    for patron in patrones_eliminar:
        texto = re.sub(patron, '', texto)
    texto = re.sub(r'[^A-Z ]', '', texto).strip()
    palabras = texto.split()
    if len(palabras) > 3:
        return ' '.join(palabras[:3])
    return texto

In [25]:
# Procesar archivo completo
marcas, modelos, autos = procesar_archivo_completo('DatosTecnicaTurboshop.xlsx')

# Guardar resultados
marcas.to_csv('marcas_completas.csv', index=False)
modelos.to_csv('modelos_completos.csv', index=False)
autos.to_csv('autos_completos.csv', index=False)

print(f"Marcas encontradas: {len(marcas)}")
print(f"Modelos encontrados: {len(modelos)}")
print(f"Autos completos: {len(autos)}")

# Mostrar algunos ejemplos
print("\nAlgunas marcas:", marcas.head().to_string())
print("\nAlgunos modelos:", modelos.head(10).to_string())
print("\nAlgunos autos completos:", autos.head().to_string())

KeyError: 'MARCA'